In [275]:
import os
import sys

# Demo dizininin içinde temp_libs klasörü oluşturalım
temp_dir = "/Users/mehmetakkan/Downloads/demo/temp_libs"
os.makedirs(temp_dir, exist_ok=True)

# Python path'e ekleyelim
if temp_dir not in sys.path:
    sys.path.append(temp_dir)

In [ ]:
# -------------- TEMEL KÜTÜPHANELER --------------
!pip install --target={temp_dir} dateutil
!pip install --target={temp_dir} pytz
!pip install --target={temp_dir} pydantic

# -------------- OPENAI KÜTÜPHANESİ --------------
!pip install --target={temp_dir} openai

# -------------- LANGCHAIN VE CREW AI KÜTÜPHANELERİ --------------
!pip install --target={temp_dir} crewai
!pip install --target={temp_dir} langchain-community
!pip install --target={temp_dir} langchain-core

# -------------- VERİ VE SORGU KÜTÜPHANELERİ --------------
!pip install --target={temp_dir} yfinance
!pip install --target={temp_dir} wikipedia
!pip install --target={temp_dir} openweathermap-api-client
!pip install --target={temp_dir} youtube-search-python
!pip install --target={temp_dir} unstructured
!pip install --target={temp_dir} langchain

In [276]:
# -------------- TEMEL KÜTÜPHANE İMPORTLARI --------------
# Temel kütüphaneler
import calendar
import pytz
from dateutil import parser
from datetime import datetime

# -------------- LANGCHAIN VE CREW AI İLİŞKİLİ -------------- 
from langchain_community.agent_toolkits.load_tools import load_tools
from pydantic import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool
from crewai import Agent, LLM, Task, Crew, Process
from crewai_tools import SerperDevTool
import requests

# -------------- WİKİPEDİA VE WEB SORGU İLİŞKİLİ ------------
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper, OpenWeatherMapAPIWrapper
from langchain_community.document_loaders import UnstructuredURLLoader, YoutubeLoader
from langchain_community.tools import YouTubeSearchTool

# -------------- LANGCHAIN CORE ARAÇLAR ------------------
from langchain_core.tools import BaseTool, StructuredTool, Tool, tool

# -------------- FİNANSAL VERİ --------------------------
import yfinance as yf

# -------------- CALLBACK YÖNETİCİLERİ ------------------
from langchain.callbacks.manager import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)



In [318]:


# -------------- API ANAHTARLARI --------------
OPENAI_API_KEY = ".."  # OpenAI API anahtarı
SERPER_API_KEY = ".." # Serper API anahtarı
OPENWEATHER_API_KEY = ".." # OpenWeather API anahtarı

# -------------- GOOGLE ARAMA ARACI --------------
def google_search(query: str):
    """
    Bitki bakımı, hastalıkları ve bakım ipuçları hakkında güncel bilgileri 
    aramak için kullanılır.
    
    Args:
        query (str): Arama sorgusu
        
    Returns:
        dict: Google arama sonuçları
    """
    headers = {
        'X-API-KEY': SERPER_API_KEY,
        'Content-Type': 'application/json'
    }
    data = {
        'q': query
    }
    response = requests.post('https://google.serper.dev/search', headers=headers, json=data)
    return response.json()

# Aracı StructuredTool olarak yapılandırma
google_search_tool = StructuredTool.from_function(
    func=google_search,
    name="google_search",
    description="""
    Bitki bakımı, hastalıkları ve bakım ipuçları hakkında güncel bilgileri 
    aramak için kullanılır. Örnek kullanımlar:
    - Belirli bir bitkinin bakım gereksinimleri
    - Yaygın bitki hastalıkları ve tedavileri
    - Mevsimsel bitki bakım tavsiyeleri
    """,
    return_direct=False
)

# -------------- WIKIPEDIA ARACI --------------
class WikipediaInput(BaseModel):
    query: str = Field(
        description="Bitki türü veya bakımı ile ilgili arama sorgusu. Örnek: 'Monstera deliciosa bakımı'"
    )

def _wikipedia_search(query: str) -> str:
    """
    Wikipedia'dan bitki ile ilgili detaylı bilgi çeker.
    
    Args:
        query (str): Aranacak bitki veya bakım konusu
        
    Returns:
        str: Wikipedia'dan alınan detaylı bilgi
    """
    wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
    return wikipedia.run(query)[: 10 * 1000]

wikipedia_search = StructuredTool.from_function(
    func=_wikipedia_search,
    name="wikipedia_search",
    description="""
    Bitkiler hakkında ansiklopedik bilgi almak için kullanılır:
    - Bitki türlerinin genel özellikleri
    - Doğal yaşam alanları ve koşulları
    - Bilimsel sınıflandırma bilgileri
    - Tarihsel ve kültürel bilgiler
    """,
    args_schema=WikipediaInput,
    return_direct=False,
)

# -------------- WEB SAYFASI OKUMA --------------
class URLInput(BaseModel):
    url: str = Field(
        description="Bitki bakımı ile ilgili web sayfası URL'si. Örnek: 'https://www.botanikamo.com/tr/blog/monstera-deve-tabani-bitkisi-bakimi?srsltid=AfmBOoqDr-uyQn1bCoKTwAZu-kyYv7GthSNWcHUijK6OgxkXw0496Ie4'"
    )

def _parse_url(url: str) -> str:
    """
    Web sayfasından bitki bakımı bilgilerini çeker.
    
    Args:
        url (str): İncelenecek web sayfasının URL'si
        
    Returns:
        str: Sayfanın işlenmiş içeriği
    """
    try:
        loader = UnstructuredURLLoader(urls=[url])
        document = loader.load()[0]
        return document.page_content
    except Exception as e:
        return f"URL okunamadı. Hata: {str(e)}"

parse_url = StructuredTool.from_function(
    func=_parse_url,
    name="parse_url",
    description="""
    Bitki bakımı ile ilgili web kaynaklarını analiz etmek için kullanılır:
    - Uzman blog yazıları
    - Bakım kılavuzları
    - Forum tartışmaları
    - Online bitki bakımı kaynakları
    """,
    args_schema=URLInput,
    return_direct=False,
)

# -------------- YOUTUBE ARAMA --------------
class QueryInput(BaseModel):
    youtube_query: str = Field(
        description="Bitki bakımı ile ilgili YouTube arama sorgusu. Örnek: 'orkide bakımı püf noktaları'"
    )

def _youtube_video_search(youtube_query: str):
    """
    Bitki bakımı ile ilgili YouTube videolarını arar.
    
    Args:
        youtube_query (str): Arama sorgusu
        
    Returns:
        str: Bulunan videoların listesi
    """
    tool = YouTubeSearchTool()
    result = tool.run(youtube_query)
    return f"Bitki bakımı ile ilgili bulunan videolar:\n{result}"

youtube_video_search = StructuredTool.from_function(
    func=_youtube_video_search,
    name="youtube_video_search",
    description="""
    Bitki bakımı ile ilgili görsel içerik bulmak için kullanılır:
    - Adım adım bakım videoları
    - Hastalık teşhis rehberleri
    - Budama ve çoğaltma teknikleri
    - Uzman bakım tavsiyeleri
    """,
    args_schema=QueryInput,
    return_direct=False,
)

# -------------- YOUTUBE TRANSKRİPT --------------
class YouTubeURLInput(BaseModel):
    youtube_url: str = Field(
        description="Bitki bakımı videosu URL'si. Örnek: 'https://www.youtube.com/watch?v=MucqGCYA2H8&ab_channel=benjiplant'"
    )

def _youtube_transcript(youtube_url):
    """
    YouTube videosunun transkriptini çıkarır.
    
    Args:
        youtube_url (str): Video URL'si
        
    Returns:
        str: Video transkripti
    """
    try:
        transcript = YoutubeLoader.from_youtube_url(
            youtube_url=youtube_url,
            language=["en", "tr", "de", "fr", "es"],
        ).load()
        return transcript[0].page_content
    except ValueError as e:
        return "Hata: Video bulunamadı veya altyazı mevcut değil"

youtube_transcript = StructuredTool.from_function(
    func=_youtube_transcript,
    name="youtube_video_transcript",
    description="""
    Bitki bakımı videolarının içeriğini metin olarak almak için kullanılır:
    - Detaylı bakım talimatları
    - Uzman önerileri ve ipuçları
    - Sorun giderme rehberleri
    - Mevsimsel bakım tavsiyeleri
    """,
    args_schema=YouTubeURLInput,
    return_direct=False,
)

# -------------- HAVA DURUMU KONTROLÜ --------------
class WeatherInput(BaseModel):
    location: str = Field(
        description="Bitkinizin bulunduğu konum. Örnek: 'Istanbul,TR' veya 'Ankara,TR'"
    )

def _check_weather(location: str) -> str:
    """
    Bitki bakımı için hava durumu bilgisi sağlar.
    
    Args:
        location (str): Konum bilgisi
        
    Returns:
        str: Hava durumu raporu
    """
    weather = OpenWeatherMapAPIWrapper(openweather_api_key=OPENWEATHER_API_KEY)
    return weather.run(location)

check_weather = StructuredTool.from_function(
    func=_check_weather,
    name="check_weather",
    description="""
    Bitki bakımı için hava koşullarını kontrol etmekte kullanılır:
    - Sıcaklık değerleri
    - Nem oranı
    - Güneşlenme durumu
    - Sulama zamanlaması için hava tahminleri
    """,
    args_schema=WeatherInput,
    return_direct=False,
)

# -------------- ARAÇLAR LİSTESİ --------------
tools = [
    google_search_tool,     # Genel bitki bilgisi araması
    wikipedia_search,       # Detaylı tür bilgisi
    check_weather,         # Bakım koşulları için hava durumu
    parse_url,             # Uzman web kaynaklarından bilgi
    youtube_video_search,  # Görsel bakım rehberleri
    youtube_transcript,    # Video içerik analizi
]

# Her araç için açıklayıcı etiketler
tool_descriptions = {
    "google_search": "Genel bitki bakımı araştırması",
    "wikipedia_search": "Bitki türü ve özellikleri bilgisi",
    "check_weather": "Bakım koşulları için hava durumu kontrolü",
    "parse_url": "Web kaynaklarından detaylı bakım bilgisi",
    "youtube_video_search": "Görsel bakım rehberleri",
    "youtube_transcript": "Video içerik analizi"
}

In [312]:
# -------------- HAVA DURUMU KONTROLÜ --------------
class WeatherInput(BaseModel):
    location: str = Field(
        description="Bitkinizin bulunduğu konum. Örnek: 'Istanbul,TR' veya 'Ankara,TR'"
    )

def _check_weather(location: str) -> str:
    """
    Bitki bakımı için hava durumu bilgisi sağlar.
    
    Args:
        location (str): Konum bilgisi
        
    Returns:
        str: Hava durumu raporu
    """
    try:
        weather_wrapper = {
            "openweathermap_api_key": OPENWEATHER_API_KEY
        }
        weather = OpenWeatherMapAPIWrapper(**weather_wrapper)
        return weather.run(location)
    except Exception as e:
        return f"Hava durumu bilgisi alınamadı: {str(e)}"

check_weather = StructuredTool.from_function(
    func=_check_weather,
    name="check_weather",
    description="""
    Bitki bakımı için hava koşullarını kontrol etmekte kullanılır:
    - Sıcaklık değerleri
    - Nem oranı
    - Güneşlenme durumu
    - Sulama zamanlaması için hava tahminleri
    """,
    args_schema=WeatherInput,
    return_direct=False
)

In [249]:
class TestBitkiBakimiTools(unittest.TestCase):
    def test_google_search(self):
        """Google arama testleri"""
        try:
            # SerperDevTool için özel kullanım
            query = "Monstera deliciosa bakımı"
            
            # _run metodunu yeniden tanımlıyoruz
            def custom_run(self, input_str):
                return f"Simulated search results for: {input_str}"
            
            # Orijinal _run metodunu saklayıp mock'u yerleştiriyoruz
            original_run = type(google_search).run
            type(google_search).run = custom_run
            
            try:
                result = google_search.run(query)
            finally:
                # Orijinal metodu geri yüklüyoruz
                type(google_search).run = original_run
            
            print(f"\nGoogle Arama Testi:")
            print(f"Sorgu: {query}")
            print(f"Sonuç: {result[:200]}...")
            print(f"Sonuç uzunluğu: {len(result)} karakter")
            
            self.assertIsNotNone(result)
            self.assertIsInstance(result, str)
            self.assertGreater(len(result), 0)
            
        except Exception as e:
            print(f"❌ Google arama hatası: {str(e)}")
            raise

    def test_openai_chat(self):
        """OpenAI API testi"""
        try:
            from openai import OpenAI
            
            client = OpenAI(api_key=OPENAI_API_KEY)
            test_prompt = "Bitki bakımı hakkında kısa bir bilgi ver."
            
            response = client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": "Sen bir bitki bakımı uzmanısın."},
                    {"role": "user", "content": test_prompt}
                ]
            )
            
            result = response.choices[0].message.content
            
            print(f"\nOpenAI Chat Testi:")
            print(f"Prompt: {test_prompt}")
            print(f"Sonuç: {result[:200]}...")
            print(f"Sonuç uzunluğu: {len(result)} karakter")
            
            self.assertIsNotNone(result)
            self.assertIsInstance(result, str)
            self.assertGreater(len(result), 0)
            
        except Exception as e:
            print(f"❌ OpenAI Chat hatası: {str(e)}")
            raise


    def test_wikipedia_search(self):
        """Wikipedia arama testleri"""
        try:
            query = "Monstera deliciosa"
            result = wikipedia_search.run(query)
            
            print(f"\nWikipedia Arama Testi:")
            print(f"Sorgu: {query}")
            print(f"Sonuç: {result[:200]}...")  # İlk 200 karakter
            print(f"Sonuç uzunluğu: {len(result)} karakter")
            
            self.assertIsNotNone(result)
            self.assertIsInstance(result, str)
            
        except Exception as e:
            print(f"❌ Wikipedia arama hatası: {str(e)}")
            raise

    def test_url_parser(self):
        """URL okuma testleri"""
        try:
            test_url = "https://www.wikipedia.org/wiki/Monstera_deliciosa"
            result = parse_url.run(test_url)
            
            print(f"\nURL Okuma Testi:")
            print(f"Test URL: {test_url}")
            print(f"Sonuç: {result[:200]}...")  # İlk 200 karakter
            print(f"Sonuç uzunluğu: {len(result)} karakter")
            
            self.assertIsNotNone(result)
            self.assertIsInstance(result, str)
            
        except Exception as e:
            print(f"❌ URL okuma hatası: {str(e)}")
            raise

    def test_youtube_search(self):
        """YouTube arama testleri"""
        try:
            query = "orkide bakımı püf noktaları"
            result = youtube_video_search.run(query)
            
            print(f"\nYouTube Arama Testi:")
            print(f"Sorgu: {query}")
            print(f"Sonuç: {result[:200]}...")  # İlk 200 karakter
            print(f"Sonuç uzunluğu: {len(result)} karakter")
            
            self.assertIsNotNone(result)
            self.assertIsInstance(result, str)
            self.assertIn("video", result.lower())
            
        except Exception as e:
            print(f"❌ YouTube arama hatası: {str(e)}")
            raise

    def test_youtube_transcript(self):
        """YouTube transkript testleri"""
        try:
            test_url = "https://www.youtube.com/watch?v=dQw4w9WgXcQ"
            result = youtube_transcript.run(test_url)
            
            print(f"\nYouTube Transkript Testi:")
            print(f"Video URL: {test_url}")
            print(f"Sonuç: {result[:200]}...")  # İlk 200 karakter
            print(f"Sonuç uzunluğu: {len(result)} karakter")
            
            self.assertIsNotNone(result)
            self.assertIsInstance(result, str)
            
        except Exception as e:
            print(f"❌ YouTube transkript hatası: {str(e)}")
            print("Not: Bu hata videonun altyazısı olmamasından kaynaklanabilir.")

    def test_weather(self):
        """Hava durumu kontrol testleri"""
        try:
            test_location = "Istanbul,TR"
            weather = OpenWeatherMapAPIWrapper(openweathermap_api_key=OPENWEATHER_API_KEY)
            result = weather.run(test_location)
            
            print(f"\nHava Durumu Testi:")
            print(f"Lokasyon: {test_location}")
            print(f"Sonuç: {result}")
            print(f"Sonuç uzunluğu: {len(result)} karakter")
            
            self.assertIsNotNone(result)
            self.assertIsInstance(result, str)
            self.assertTrue(any(keyword in result.lower() for keyword in ['sıcaklık', 'derece', 'temp']))
            
        except Exception as e:
            print(f"❌ Hava durumu kontrolü hatası: {str(e)}")
            raise

def run_all_tests():
    """Tüm testleri çalıştır ve sonuçları raporla"""
    print("\n🌿 BİTKİ BAKIMI ARAÇLARI TEST RAPORU 🌿")
    print("=" * 50)
    
    test_suite = unittest.TestLoader().loadTestsFromTestCase(TestBitkiBakimiTools)
    test_result = unittest.TextTestRunner(verbosity=2).run(test_suite)
    
    print("\n📊 TEST SONUÇLARI:")
    print(f"Toplam Test Sayısı: {test_result.testsRun}")
    print(f"Başarılı Testler: {test_result.testsRun - len(test_result.failures) - len(test_result.errors)} ✅")
    print(f"Başarısız Testler: {len(test_result.failures)} ❌")
    print(f"Hatalı Testler: {len(test_result.errors)} ⚠️")

if __name__ == '__main__':
    run_all_tests()

test_google_search (__main__.TestBitkiBakimiTools.test_google_search)
Google arama testleri ... ok
test_openai_chat (__main__.TestBitkiBakimiTools.test_openai_chat)
OpenAI API testi ... 


🌿 BİTKİ BAKIMI ARAÇLARI TEST RAPORU 🌿

Google Arama Testi:
Sorgu: Monstera deliciosa bakımı
Sonuç: Simulated search results for: Monstera deliciosa bakımı...
Sonuç uzunluğu: 55 karakter


ok
test_url_parser (__main__.TestBitkiBakimiTools.test_url_parser)
URL okuma testleri ... 


OpenAI Chat Testi:
Prompt: Bitki bakımı hakkında kısa bir bilgi ver.
Sonuç: Bitki bakımı, bitkilerin sağlıklı ve güçlü bir şekilde büyümesi için gerekli olan düzenli sulama, doğru gübreleme, uygun ışık ve sıcaklık koşulları sağlama gibi işlemleri kapsar. Her bitki türü farklı...
Sonuç uzunluğu: 679 karakter


ok
test_weather (__main__.TestBitkiBakimiTools.test_weather)
Hava durumu kontrol testleri ... 

...


Wikipedia Arama Testi:
Sorgu: Monstera deliciosa
Sonuç: Page: Monstera deliciosa
Summary: Monstera deliciosa, the Swiss cheese plant or split-leaf philodendron is a species of flowering plant native to tropical forests of southern Mexico, south to Panama. ...
Sonuç uzunluğu: 1779 karakter


ok
test_youtube_transcript (__main__.TestBitkiBakimiTools.test_youtube_transcript)
YouTube transkript testleri ... 


YouTube Arama Testi:
Sorgu: orkide bakımı püf noktaları
Sonuç: Bitki bakımı ile ilgili bulunan videolar:
['https://www.youtube.com/watch?v=393YAUS22B4&pp=ygUfb3JraWRlIGJha8SxbcSxIHDDvGYgbm9rdGFsYXLEsQ%3D%3D', 'https://www.youtube.com/watch?v=PtT-4UlCZYg&pp=ygUfb3...
Sonuç uzunluğu: 248 karakter


ok

----------------------------------------------------------------------
Ran 7 tests in 10.035s

OK



YouTube Transkript Testi:
Video URL: https://www.youtube.com/watch?v=dQw4w9WgXcQ
Sonuç: [Music] we're no strangers to love you know the rules and so do I I full commitments while I'm thinking of you wouldn't get this from any other guy I just want to tell you how I'm feeling got to make ...
Sonuç uzunluğu: 1848 karakter

📊 TEST SONUÇLARI:
Toplam Test Sayısı: 7
Başarılı Testler: 7 ✅
Başarısız Testler: 0 ❌
Hatalı Testler: 0 ⚠️


In [313]:
# -------------- YAPAY ZEKA MODELİ KURULUMU --------------
llm = LLM(
    model="gpt-3.5-turbo", 
    temperature=0.7,
    api_key=OPENAI_API_KEY,
    verbose=True
)

In [314]:
# -------------- BİTKİ UZMANI AGENT --------------
plant_expert = Agent(
    role="Bitki Ansiklopedi Uzmanı",
    goal="""Kullanıcının bitkileri hakkında detaylı ve doğru bilgi sağlamak, 
    türleri doğru tanımlamak ve temel bakım bilgilerini araştırmak""",
    backstory="""Sen deneyimli bir botanik uzmanı ve bitki ansiklopedisisin. 
    30 yıllık deneyiminle binlerce bitki türünü tanıyabilir, 
    her birinin özelliklerini, kökenini ve temel bakım ihtiyaçlarını bilirsin. 
    Wikipedia ve güvenilir kaynaklardan edindiğin bilgileri kullanarak 
    her bitkinin doğal yaşam koşullarını ve temel bakım gereksinimlerini açıklarsın.""".strip(),
    tools=[],  # Tüm araçlara erişimi var
    llm=llm
)

plant_expert.tools = tools

# -------------- BAKIM KOÇU AGENT --------------
care_coach = Agent(
    role="Kişisel Bitki Bakım Koçu",
    goal="""Bitkiler için kişiselleştirilmiş bakım planları oluşturmak, 
    günlük/haftalık/aylık bakım programları hazırlamak ve 
    kullanıcıya özel bakım tavsiyeleri vermek""",
    backstory="""Sen profesyonel bir bitki bakım danışmanısın. 
    Hava koşullarını, mevsimleri ve bitkinin bulunduğu ortamı değerlendirerek 
    en uygun bakım programlarını oluşturursun. 
    Her bitkinin özel ihtiyaçlarını anlar, 
    kullanıcının yaşam tarzına uygun bakım rutinleri önerirsin. 
    YouTube'daki uzman videolarından da faydalanarak 
    görsel destekli bakım tavsiyeleri sunarsın.""".strip(),
    tools=[],  # Sadece ilgili araçlar
    llm=llm
)

care_coach.tools = [check_weather, youtube_video_search, youtube_transcript]

# -------------- SORUN ÇÖZÜCÜ AGENT --------------
problem_solver = Agent(
    role="Bitki Sağlığı Uzmanı",
    goal="""Bitki sağlığı sorunlarını teşhis etmek, 
    çözüm önerileri sunmak ve önleyici tedbirler konusunda danışmanlık yapmak""",
    backstory="""Sen uzman bir bitki patoloğu ve tedavi uzmanısın. 
    20 yıllık deneyiminle bitki hastalıklarını, 
    zararlıları ve bakım kaynaklı sorunları teşhis edebilirsin. 
    Her soruna bilimsel ve pratik çözümler üretir, 
    acil durumlarda hızlı müdahale yöntemleri önerirsin. 
    Google ve uzman kaynaklardan en güncel tedavi yöntemlerini araştırır, 
    sorunları kökünden çözmek için kapsamlı tedavi planları hazırlarsın.""".strip(),
    tools=[],  # Tüm araçlara erişimi var
    llm=llm
)

problem_solver.tools = tools

In [315]:
# -------------- BİTKİ TANIMLAMA VE ANALİZ GÖREVİ --------------
plant_analysis_task = Task(
    description="""
        `{plant_name}` bitkisi için detaylı bir analiz raporu hazırla:
        
        1. Bitkinin türü ve özellikleri hakkında ansiklopedik bilgi topla
        2. Doğal yaşam koşullarını araştır
        3. Temel bakım gereksinimlerini belirle
        4. Yaygın sorunlarını listele
        
        YouTube ve web kaynaklarından görsel destekli bilgiler ekle.
        Wikipedia'dan bilimsel bilgileri dahil et.
    """.strip(),
    expected_output="""
        Detaylı bitki analiz raporu:
        - Tür bilgisi ve özellikleri
        - Doğal yaşam koşulları
        - Temel bakım gereksinimleri
        - Potansiyel sorunlar
        Kullanılan kaynakların listesi ile birlikte.
    """,
    agent=plant_expert,
    tools=[]
)

plant_analysis_task.tools = tools

# -------------- BAKIM PLANI OLUŞTURMA GÖREVİ --------------
care_plan_task = Task(
    description="""
        `{plant_name}` bitkisi için kişiselleştirilmiş bir bakım planı hazırla:
        
        1. `{location}` için hava durumunu kontrol et
        2. Günlük, haftalık ve aylık bakım takvimi oluştur
        3. Sulama, gübreleme ve budama programı belirle
        4. Mevsimsel bakım önerileri sun
        
        YouTube'dan uzman bakım videoları ekle.
        Hava koşullarına göre bakım önerilerini güncelle.
    """.strip(),
    expected_output="""
        Kişiselleştirilmiş bakım planı:
        - Günlük/Haftalık/Aylık yapılacaklar
        - Sulama ve gübreleme programı
        - Mevsimsel bakım tavsiyeleri
        - Önerilen video kaynakları
    """,
    agent=care_coach,
    tools=[]
)

care_plan_task.tools = [check_weather, youtube_video_search, youtube_transcript]

# -------------- SORUN TEŞHİS VE ÇÖZÜM GÖREVİ --------------
problem_solving_task = Task(
    description="""
        `{plant_name}` bitkisinin `{problem_description}` sorunu için çözüm planı hazırla:
        
        1. Sorunun kök nedenini araştır
        2. Benzer vaka çözümlerini incele
        3. Acil müdahale planı oluştur
        4. Uzun vadeli tedavi önerileri sun
        
        Google'dan güncel tedavi yöntemlerini araştır.
        Uzman kaynaklardan çözüm önerilerini derle.
    """.strip(),
    expected_output="""
        Sorun çözüm raporu:
        - Sorunun teşhisi ve nedenleri
        - Acil müdahale adımları
        - Uzun vadeli tedavi planı
        - Önleyici tedbirler
        Kullanılan kaynaklar ve referanslar ile birlikte.
    """,
    agent=problem_solver,
    tools=[]
)

problem_solving_task.tools = tools

In [317]:
# -------------- CREW OLUŞTURMA --------------
crew = Crew(
    agents=[
        plant_expert,      # Bitki Ansiklopedi Uzmanı
        care_coach,        # Kişisel Bitki Bakım Koçu
        problem_solver     # Bitki Sağlığı Uzmanı
    ],
    tasks=[
        plant_analysis_task,    # Bitki analiz görevi
        care_plan_task,         # Bakım planı oluşturma görevi
        problem_solving_task    # Sorun çözme görevi
    ],
    process=Process.sequential,  # Görevleri sırayla işle
    verbose=True,               # Detaylı log tut
    allow_delegation=True,      # Agent'ların birbirine görev devretmesine izin ver
)

# -------------- ÖRNEK KULLANIM --------------
# Sistemi çalıştırma örneği
result = crew.kickoff(
    inputs={
        "plant_name": "Aloe Vera",     # Örnek bitki türü
        "location": "Istanbul,TR",                  # Örnek konum
        "problem_description": "yapraklarda sararma"    # Örnek sorun
    }
)

# -------------- SONUÇ FORMATLAMA --------------
def format_results(crew_result):
    """
    Crew'dan gelen sonuçları düzenli bir formatta sunar.
    
    Args:
        crew_result: Crew'dan gelen ham sonuç
        
    Returns:
        str: Formatlanmış sonuç raporu
    """
    return f"""
    🌿 BİTKİ BAKIM RAPORU 🌿
    ========================
    
    1. BİTKİ ANALİZİ
    ----------------
    {crew_result['plant_analysis_task']}
    
    2. BAKIM PLANI
    --------------
    {crew_result['care_plan_task']}
    
    3. SORUN ÇÖZÜMÜ
    --------------
    {crew_result['problem_solving_task']}
    
    ⚠️ Not: Bu rapor yapay zeka destekli bir sistem tarafından oluşturulmuştur.
    Kritik kararlar için lütfen bir bitki uzmanına danışınız.
    """

# -------------- SİSTEM KULLANIMI --------------
"""
Sistem şu şekilde kullanılabilir:

1. Yeni bir bitki için:
crew.kickoff(inputs={
    "bitki_turu": "Orkide",
    "konum": "Ankara,TR",
    "sorun_tanimi": ""
})

2. Sorun yaşanan bir bitki için:
crew.kickoff(inputs={
    "bitki_turu": "Kaktüs",
    "konum": "Izmir,TR",
    "sorun_tanimi": "aşırı sulama sonrası kökler çürümeye başladı"
})

3. Sadece bakım planı için:
crew.kickoff(inputs={
    "bitki_turu": "Aloe Vera",
    "konum": "Antalya,TR",
    "sorun_tanimi": ""
})
"""

2024-11-12 13:21:18,557 - 8475293760 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


# Agent: Bitki Ansiklopedi Uzmanı
## Task: `Aloe Vera` bitkisi için detaylı bir analiz raporu hazırla:
        
        1. Bitkinin türü ve özellikleri hakkında ansiklopedik bilgi topla
        2. Doğal yaşam koşullarını araştır
        3. Temel bakım gereksinimlerini belirle
        4. Yaygın sorunlarını listele
        
        YouTube ve web kaynaklarından görsel destekli bilgiler ekle.
        Wikipedia'dan bilimsel bilgileri dahil et.


# Agent: Bitki Ansiklopedi Uzmanı
## Thought: Aloe Vera bitkisi için detaylı bir analiz raporu oluşturmak için Wikipedia'dan bilimsel bilgileri araştırmalı ve doğal yaşam koşulları ile temel bakım gereksinimlerini belirlemeliyim. Ayrıca yaygın sorunları öğrenmek için web kaynaklarından da faydalanmalıyım.
## Using tool: wikipedia_search
## Tool Input: 
"{\"query\": \"Aloe Vera\"}"
## Tool Output: 
Page: Aloe vera
Summary: Aloe vera () is a succulent plant species of the genus Aloe. It is widely distributed, and is considered an invasive species in 

'\nSistem şu şekilde kullanılabilir:\n\n1. Yeni bir bitki için:\ncrew.kickoff(inputs={\n    "bitki_turu": "Orkide",\n    "konum": "Ankara,TR",\n    "sorun_tanimi": ""\n})\n\n2. Sorun yaşanan bir bitki için:\ncrew.kickoff(inputs={\n    "bitki_turu": "Kaktüs",\n    "konum": "Izmir,TR",\n    "sorun_tanimi": "aşırı sulama sonrası kökler çürümeye başladı"\n})\n\n3. Sadece bakım planı için:\ncrew.kickoff(inputs={\n    "bitki_turu": "Aloe Vera",\n    "konum": "Antalya,TR",\n    "sorun_tanimi": ""\n})\n'